In [62]:
import labelbox as lb
from dotenv import load_dotenv, find_dotenv
import random

_=load_dotenv(find_dotenv())

client = lb.Client()

model_name = "Animal_Model"

client.enable_experimental = True


In [63]:
model = client.get_models(where=(lb.Model.name == model_name)).get_one()
if not model:
    proj: lb.Project = client.get_project("clp7euycs059o0703gmcug1re")
    ontology: lb.Ontology = proj.ontology()
    model = client.create_model(
        name=model_name,
        ontology_id=ontology.uid,
        )

In [64]:
model_run_name = "model_run_1"
model_run = ""

In [65]:
model_runs: list[lb.ModelRun] = model.model_runs()
exists = False
for model_run_check in model_runs:
    if model_run_check.name == model_run_name:
        model_run = model_run_check
        exists = True
        break

if not exists:
    model_run = model.create_model_run(
        name=model_run_name,
        config= {
            "image_size": 32,
            "activation": "relu",
            "optimizer": "adam"
        }
    )

In [66]:
print(model_run.name)
print(model_run.get_config())

model_run_1
{'optimizer': 'adam', 'activation': 'relu', 'image_size': 32}


In [78]:
batches: list[lb.Batch] = proj.batches()

train = []
test = []
validation = []
data_row_ids = []

for batch in batches:
    data_rows: list[lb.DataRow] = batch.export_data_rows(include_metadata=True)
    global_keys = []
    for data_row in list(data_rows):
        global_keys.append(data_row.global_key)
        data_row_ids.append(data_row.uid)
    random.shuffle(global_keys)
    for i, global_key in enumerate(global_keys[:300]):
        test.append(global_key)
        global_keys.pop(i)
    for i, global_key in enumerate(global_keys[:100]):
        validation.append(global_key)
        global_keys.pop(i)
    train.extend(global_keys)

print(len(train), len(test), len(validation))

    
    

22179 3000 1000


In [81]:
model_run.upsert_labels(project_id=proj.uid)

True

In [84]:
client.enable_experimental = True
print(len(train), len(test), len(validation))

model_run.assign_data_rows_to_split(
    global_keys=train,
    split="TRAINING",
)

model_run.assign_data_rows_to_split(
    global_keys=test,
    split="TEST",
)

model_run.assign_data_rows_to_split(
    global_keys=validation,
    split="VALIDATION",
)

22179 3000 1000


True